### 3. Analýza známek u studijní skupiny (gql_ug + gql_granting).

Jako parametr vezme definici kontingenční tabulky, GQL dotaz, dotaz zrealizuje, a data vloží do sešitu Excel spolu s kontingenční tabulkou. Vytvořte jako FastAPI službu.

Společné podmínky
- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf
- Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

In [5]:
import aiohttp
import asyncio
import json

In [2]:
gql_query = """
query {
	acClassificationPage {
  	id
  	student {
    	id
    	email
    	fullname
    	name
    	surname
  	}
  	semester {
    	id
    	order
    	subject {
      	id
      	name
      	program {
        	id
        	name
      	}
    	}
  	}
  	level {
    	id
    	name
  	}
		date
  	order
  }
}
"""

In [3]:
with open('data.json', 'r') as file:
    data = json.load(file)
    
print(data)

{'data': {'acClassificationPage': [{'id': 'ce250bd0-b095-11ed-9bd8-0242ac110002', 'student': {'id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'email': 'john.newbie@world.com', 'fullname': 'John Newbie', 'name': 'John', 'surname': 'Newbie'}, 'semester': {'id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'order': 1, 'subject': {'id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'name': 'Programovani', 'program': {'id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'name': 'IT Technologie'}}}, 'level': {'id': '5faea396-b095-11ed-9bd8-0242ac110002', 'name': 'F'}, 'date': '2023-04-19T08:00:00', 'order': 1}, {'id': 'ce250bd1-b095-11ed-9bd8-0242ac110002', 'student': {'id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'email': 'john.newbie@world.com', 'fullname': 'John Newbie', 'name': 'John', 'surname': 'Newbie'}, 'semester': {'id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'order': 1, 'subject': {'id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'name': 'Programovani', 'program': {'id': '2766fc9a-b095-11ed-9bd8-0242ac1

In [4]:
def convertLevel(level):
    row = {}
    
    row["level_id"] = level["level"]["id"]
    row["level_name"] = level["level"]["name"]
    row["level_order"] = level["order"]
    row["level_date"] = level["date"]
    
    row["student_id"] = level["student"]["id"]  
    row["student_email"] = level["student"]["email"]  
    
    row["subject_id"] = level["semester"]["subject"]["id"]  
    row["subject_name"] = level["semester"]["subject"]["name"]   
    row["semestr_order"] = level['semester']['order']
    
    row["program_id"] = level["semester"]["subject"]["program"]["id"]  
    row["program_name"] = level["semester"]["subject"]["program"]["name"]
    return row

converted_data = [convertLevel(level) for level in data["data"]["acClassificationPage"]]

sorted_data = sorted(converted_data, key=lambda x: x['level_order'], reverse=True)
filtered_data = sorted_data[:1]

print(filtered_data)

[{'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'level_date': '2023-04-19T08:00:00', 'student_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'student_email': 'john.newbie@world.com', 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programovani', 'semestr_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}]


In [5]:
with open('fake_data.json', 'r') as file:
    fake_data = json.load(file)
    
grouped_data = filtered_data + fake_data

print(grouped_data)

with open("grouped_data.json", "w") as json_file:
    json.dump(grouped_data, json_file)

[{'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'level_date': '2023-04-19T08:00:00', 'student_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'student_email': 'john.newbie@world.com', 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programovani', 'semestr_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}, {'level_id': '6d5a1c9e-3db6-11ec-8d3d-0242ac130003', 'level_name': 'A', 'level_order': 3, 'level_date': '2023-05-20T09:30:00', 'student_id': '4f8e72aa-3db6-11ec-8d3d-0242ac130003', 'student_email': 'emma.smith@example.com', 'subject_id': 'ce250a69-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Pocitacove site', 'semester_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}, {'level_id': '7e6b3f8a-3db6-11ec-8d3d-0242ac130003', 'level_name': 'B', 'level_order': 4, 'level_date': '2023-06-10T14:00:00', 'student_id': '5fbc249e-3db6-11

</br>

### Nový dotaz

In [1]:
import aiohttp
import asyncio
import json

In [2]:
# nový dotaz, kde jdeme přes skupiny ne uživatele
# místo grouppage za name můžeme přidat groupbyid (tak docílíme, že dostaneme jen data od jedné skupiny)

gql_query = """
guery {
  #groupPage{
  groupById(id: "2d9dcd22-a4a2-11ed-b9df-0242ac120003"){
    name
    id
    memberships{
      user{
        id
        fullname
        classifications{
          level{
            id
            name
          }
          id
          order
          semester{
            id
            order
            subject{
              id
              name
            }
          }
        }
      }
    }
  }
}
"""

In [3]:
# data z druhého dotazu

with open('data2.json', 'r') as file:
    data = json.load(file)
    
print(data)

{'data': {'groupById': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni', 'memberships': [{'user': {'id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'fullname': 'John Newbie', 'classifications': [{'level': {'id': '5faea396-b095-11ed-9bd8-0242ac110002', 'name': 'F'}, 'id': 'ce250bd0-b095-11ed-9bd8-0242ac110002', 'order': 1, 'semester': {'id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'order': 1, 'subject': {'id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'name': 'Programování'}}}, {'level': {'id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'name': 'C'}, 'id': 'ce250bd1-b095-11ed-9bd8-0242ac110002', 'order': 2, 'semester': {'id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'order': 1, 'subject': {'id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'name': 'Programování'}}}]}}]}}}


In [7]:
import json

with open('data2.json', 'r') as file:
    data = json.load(file)
    
def convertLevel(membership):
    user = membership["user"]
    classifications = user["classifications"]
    
    rows = []
    for classification in classifications:
        row = {}
        row["user_fullname"] = membership["user"]["fullname"]
        row["user_id"] = user["id"]
        
        row["level_id"] = classification["level"]["id"]
        row["level_name"] = classification["level"]["name"]
        row["level_order"] = classification["order"]
        
        row["subject_id"] = classification["semester"]["subject"]["id"]
        row["subject_name"] = classification["semester"]["subject"]["name"]
        
        row["semester_id"] = classification["semester"]["id"]
        row["semester_order"] = classification["semester"]["order"]
        rows.append(row)    
    return rows

memberships = data["data"]["groupById"]["memberships"]
converted_data = [convertLevel(membership) for membership in memberships]

flattened_data = [item for sublist in converted_data for item in sublist]

sorted_data = sorted(flattened_data, key=lambda x: x['level_order'], reverse=True)

filtered_data = sorted_data[:1]

print(filtered_data)

[{'user_fullname': 'John Newbie', 'user_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programování', 'semester_id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'semester_order': 1}]


</br>
</br>
</br>
</br>
</br>
</br>

### Autentizace

In [16]:
import aiohttp

async def getToken(username, password):
    keyurl = "http://host.docker.internal:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            # print(resp.status)
            keyJson = await resp.json()
            # print(keyJson)
        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            # print(resp.status)
            tokenJson = await resp.json()
            # print(tokenJson)
    return tokenJson.get("token", None)

def query(q, token):
    async def post(variables):
        gqlurl = "http://host.docker.internal:33001/api/gql"
        payload = {"query": q, "variables": variables}
        # headers = {"Authorization": f"Bearer {token}"}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            # print(headers, cookies)
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                # print(resp.status)
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

In [17]:
from itertools import product
from functools import reduce

def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            # print("flatList", row)
            yield row

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    # print("flatDict.result", result)
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)

    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

In [18]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"


### TODO:

queryStr = """
{
  result: userPage(
    where: {memberships: {group: {name: {_ilike: "%uni%"}}}}
  ) {
    id
    email
    name
    surname
    presences {
      event {
        id
        name
        startdate
        enddate
        eventType {
          id
          name
        }
      }
      presenceType {
        id
        name
      }
    }
  }
}
"""

### TODO:

mappers = {
    "id": "id",
    "name": "name",
    "surname": "surname",
    "email": "email",
    "eventid": "presences.event.id",
    "eventname": "presences.event.name",
    # "eventduration": "presences.event.duration",
    "startdate": "presences.event.startdate",
    "enddate": "presences.event.enddate",
    "eventTypeid": "presences.event.eventType.id",
    "eventTypename": "presences.event.eventType.name",
    "presenceTypeid": "presences.presenceType.id",
    "presenceTypename": "presences.presenceType.name"
}

import pandas as pd
def toTable(data):
    return pd.DataFrame(data) 

async def fullPipe():
    global pandasData
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("result", None)

    flatData = flatten(result, {}, mappers)
    pandasData = toTable(flatData)
    return pandasData

await fullPipe()

# print(list(flatData))

,id,name,surname,email,eventid,eventname,startdate,enddate,eventTypeid,eventTypename,presenceTypeid,presenceTypename
0,89d1e724-ae0f-11ed-9bd8-0242ac110002,Jana,Newbie,jana.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
1,89d1f34a-ae0f-11ed-9bd8-0242ac110002,Jolana,Newbie,jolana.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
2,89d1f3cc-ae0f-11ed-9bd8-0242ac110002,Jitka,Newbie,jitka.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
3,89d1f430-ae0f-11ed-9bd8-0242ac110002,Jaroslava,Newbie,jaroslava.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
4,89d1f48a-ae0f-11ed-9bd8-0242ac110002,Lada,Newbie,lada.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
5,89d1f4e4-ae0f-11ed-9bd8-0242ac110002,Ludmila,Newbie,ludmila.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
6,89d1f534-ae0f-11ed-9bd8-0242ac110002,Lucie,Newbie,lucie.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
7,89d1f58e-ae0f-11ed-9bd8-0242ac110002,Nola,Newbie,nola.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
8,89d1f5de-ae0f-11ed-9bd8-0242ac110002,Neva,Newbie,neva.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
9,89d1f638-ae0f-11ed-9bd8-0242ac110002,Nora,Newbie,nora.newbie@world.com,45b2df80-ae0f-11ed-9bd8-0242ac110002,Zkouška,2023-04-19T08:00:00,2023-04-19T09:00:00,b87d3ff0-8fd4-11ed-a6d4-0242ac110002,Ostatní,466398c6-a79c-11ed-b76e-0242ac110002,Přítomen
